In [1]:
import numpy as np
import cv2 as cv
import os

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix

import pickle


In [2]:
class Dictionary(object):
    def __init__(self, name, img_filenames, num_words):
        self.name = name #name of your dictionary
        self.img_filenames = img_filenames #list of image filenames
        self.num_words = num_words #the number of words
        self.training_data = [] #this is the training data required by the K-Means algorithm
        self.words = [] #list of words, which are the centroids of clusters
    def learn(self):
        sift = cv.xfeatures2d.SIFT_create()
        num_keypoints = [] #this is used to store the number of keypoints in each image
        #load training images and compute SIFT descriptors
        for filename in self.img_filenames:
            img = cv.imread(filename)
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            list_des = sift.detectAndCompute(img_gray, None)[1]
            if list_des is None:
                num_keypoints.append(0)
            else:
                num_keypoints.append(len(list_des))
                for des in list_des:
                    self.training_data.append(des)
        #cluster SIFT descriptors using K-means algorithm
        kmeans = KMeans(self.num_words)
        kmeans.fit(self.training_data)
        self.words = kmeans.cluster_centers_
        #create word histograms for training images
        training_word_histograms = [] #list of word histograms of all training images
        index = 0
        for i in range(0, len(self.img_filenames)):
            #for each file, create a histogram
            histogram = np.zeros(self.num_words, np.float32)
            #if some keypoints exist
            if num_keypoints[i] > 0:
                for j in range(0, num_keypoints[i]):
                    histogram[kmeans.labels_[j + index]] += 1
                index += num_keypoints[i]
                histogram /= num_keypoints[i]
                training_word_histograms.append(histogram)
        return training_word_histograms
    def create_word_histograms(self, img_filenames):
        sift = cv.xfeatures2d.SIFT_create()
        histograms = []
        for filename in img_filenames:
            img = cv.imread(filename)
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            descriptors = sift.detectAndCompute(img_gray, None)[1]
            histogram = np.zeros(self.num_words, np.float32) #word histogram for the input image
            if descriptors is not None:
                for des in descriptors:
                #find the best matching word
                    min_distance = 1111111 #this can be any large number
                    matching_word_ID = -1 #initial matching_word_ID=-1 means no matching
                
                    for i in range(0, self.num_words): #search for the best matching word
                        distance = np.linalg.norm(des - self.words[i])
                        if distance < min_distance:
                            min_distance = distance
                            matching_word_ID = i
                    histogram[matching_word_ID] += 1
                histogram /= len(descriptors) #normalise histogram to frequencies
            histograms.append(histogram)
        return histograms

In [3]:
foods = ['Cakes', 'Pasta', 'Pizza']
path = 'FoodImages/'
training_file_names = []
training_food_labels = []
for i in range(0, len(foods)):
    sub_path = path + 'Train/' + foods[i] + '/'
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)]
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i
    training_file_names += sub_file_names
    training_food_labels += sub_food_labels
#print(training_file_names)
#print(training_food_labels)

In [4]:
num_words = 50
dictionary_name = 'food'
dictionary = Dictionary(dictionary_name, training_file_names, num_words)

In [5]:
training_word_histograms = dictionary.learn()

In [6]:
#save dictionary
with open('food_dictionary.dic', 'wb') as f: #'wb' is for binary write
    pickle.dump(dictionary, f)

In [7]:
with open('food_dictionary.dic', 'rb') as f: #'rb' is for binary read
    dictionary = pickle.load(f)

In [8]:
test_file_names = []
test_food_labels = []
for i in range(0, len(foods)):
    sub_path = path + 'Test/' + foods[i] + '/'
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)]
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i
    test_file_names += sub_file_names
    test_food_labels += sub_food_labels

word_histograms = dictionary.create_word_histograms(test_file_names)

In [9]:
k_list = [10,15,20,25,30]

max_accuracy = 0
best_k = k_list[0]

for num_nearest_neighbours in k_list:
    knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
    knn.fit(training_word_histograms, training_food_labels)
    
    #test knn on all test images
    predicted_food_labels = knn.predict(word_histograms)
    #evaluation for accuracy and displayying confusion matrix
    num_correct_predictions = np.sum(predicted_food_labels == test_food_labels)
    recognition_acc = num_correct_predictions / (len(test_food_labels))
    print("Correct predictions for k", num_nearest_neighbours, "is", num_correct_predictions)
    print("Accuracy for k", num_nearest_neighbours, "is", recognition_acc)
    cm = confusion_matrix(test_food_labels, predicted_food_labels)
    
    if (recognition_acc > max_accuracy):
        max_accuracy = recognition_acc
        best_k = num_nearest_neighbours
        
    print("Confusion Matrix:\n", cm)
    print ("######################################################")

Correct predictions for k 10 is 63
Accuracy for k 10 is 0.7
Confusion Matrix:
 [[15  6  9]
 [ 0 27  3]
 [ 0  9 21]]
######################################################
Correct predictions for k 15 is 61
Accuracy for k 15 is 0.6777777777777778
Confusion Matrix:
 [[14  6 10]
 [ 0 26  4]
 [ 0  9 21]]
######################################################
Correct predictions for k 20 is 64
Accuracy for k 20 is 0.7111111111111111
Confusion Matrix:
 [[14  9  7]
 [ 0 27  3]
 [ 0  7 23]]
######################################################
Correct predictions for k 25 is 60
Accuracy for k 25 is 0.6666666666666666
Confusion Matrix:
 [[14 10  6]
 [ 0 26  4]
 [ 0 10 20]]
######################################################
Correct predictions for k 30 is 59
Accuracy for k 30 is 0.6555555555555556
Confusion Matrix:
 [[11 13  6]
 [ 0 26  4]
 [ 0  8 22]]
######################################################


In [10]:
c_list = [10, 20, 30, 40, 50]
max_accuracy = 0
best_c_val = c_list[0]

for c_val in c_list:
    print ("Training SVM using C =", c_val)
    svm_classifier = svm.SVC(C = c_val, #see slide 32 in week 4 lecture slides
                             kernel = 'linear') #see slide 35 in week 4 lecture slides
    svm_classifier.fit(training_word_histograms, training_food_labels)   
    predicted_food_labels = svm_classifier.predict(word_histograms)
    #evaluation for accuracy and displayying confusion matrix
    num_correct_predictions = np.sum(predicted_food_labels == test_food_labels)
    recognition_acc = num_correct_predictions / (len(test_food_labels))
    print("Correct predictions for c", c_val, "is", num_correct_predictions)
    print("Accuracy for c", c_val, "is", recognition_acc)
    cm = confusion_matrix(test_food_labels, predicted_food_labels)
    
    if (recognition_acc > max_accuracy):
        max_accuracy = recognition_acc
        best_c_val = c_val
        
    print("Confusion Matrix:\n", cm)
    print ("######################################################")

Training SVM using C = 10
Correct predictions for c 10 is 69
Accuracy for c 10 is 0.7666666666666667
Confusion Matrix:
 [[25  2  3]
 [ 0 24  6]
 [ 1  9 20]]
######################################################
Training SVM using C = 20
Correct predictions for c 20 is 76
Accuracy for c 20 is 0.8444444444444444
Confusion Matrix:
 [[26  2  2]
 [ 0 23  7]
 [ 1  2 27]]
######################################################
Training SVM using C = 30
Correct predictions for c 30 is 74
Accuracy for c 30 is 0.8222222222222222
Confusion Matrix:
 [[26  2  2]
 [ 0 23  7]
 [ 1  4 25]]
######################################################
Training SVM using C = 40
Correct predictions for c 40 is 75
Accuracy for c 40 is 0.8333333333333334
Confusion Matrix:
 [[26  2  2]
 [ 0 23  7]
 [ 1  3 26]]
######################################################
Training SVM using C = 50
Correct predictions for c 50 is 75
Accuracy for c 50 is 0.8333333333333334
Confusion Matrix:
 [[26  2  2]
 [ 0 23  7]
 [ 1  3 

In [11]:
n_estimators_list = [50, 100, 150, 200, 250]
max_accuracy = 0
best_n_estimators = n_estimators_list[0]

for n_estimators in n_estimators_list:
    adb_classifier = AdaBoostClassifier(n_estimators = n_estimators, random_state = 42)
    adb_classifier.fit(training_word_histograms, training_food_labels)
    predicted_food_labels = adb_classifier.predict(word_histograms)
    #evaluation for accuracy and displayying confusion matrix
    num_correct_predictions = np.sum(predicted_food_labels == test_food_labels)
    recognition_acc = num_correct_predictions / (len(test_food_labels))
    print("Correct predictions for n_estimators", n_estimators, "is", num_correct_predictions)
    print("Accuracy for n_estimators", n_estimators, "is", recognition_acc)
    cm = confusion_matrix(test_food_labels, predicted_food_labels)
    
    if (recognition_acc > max_accuracy):
        max_accuracy = recognition_acc
        best_n_estimators = n_estimators
        
    print("Confusion Matrix:\n", cm)
    print ("######################################################")

Correct predictions for n_estimators 50 is 66
Accuracy for n_estimators 50 is 0.7333333333333333
Confusion Matrix:
 [[21  6  3]
 [ 0 25  5]
 [ 2  8 20]]
######################################################
Correct predictions for n_estimators 100 is 59
Accuracy for n_estimators 100 is 0.6555555555555556
Confusion Matrix:
 [[17 11  2]
 [ 0 24  6]
 [ 2 10 18]]
######################################################
Correct predictions for n_estimators 150 is 63
Accuracy for n_estimators 150 is 0.7
Confusion Matrix:
 [[17  5  8]
 [ 0 23  7]
 [ 2  5 23]]
######################################################
Correct predictions for n_estimators 200 is 64
Accuracy for n_estimators 200 is 0.7111111111111111
Confusion Matrix:
 [[20  6  4]
 [ 0 23  7]
 [ 2  7 21]]
######################################################
Correct predictions for n_estimators 250 is 64
Accuracy for n_estimators 250 is 0.7111111111111111
Confusion Matrix:
 [[18  7  5]
 [ 0 23  7]
 [ 1  6 23]]
######################

In [12]:
print ("The best k for KNN is: ", best_k)
print ("The best c for SVM is: ", best_c_val)
print ("The best n_estimators for AdaBoostClassifier is: ", best_n_estimators)


The best k for KNN is:  20
The best c for SVM is:  20
The best n_estimators for AdaBoostClassifier is:  50
